In [3]:
from __future__ import division, absolute_import, print_function
import os, sys, time, re, json
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
import StringIO
import json
import cv2

imread = plt.imread
def imread8(im_file):
    ''' Read image as a 8-bit numpy array '''
    im = np.asarray(Image.open(im_file))
    return im

def read_png(res):
    img = Image.open(StringIO.StringIO(res))
    return np.array(img)

def read_npy(res):
    return np.load(res)

# Matches a color from the object_mask and then returns that region color
def match_color(object_mask, target_color, tolerance=3):
    match_region = np.ones(object_mask.shape[0:2], dtype=bool)
    for c in range(3): # r,g,b
        min_val = target_color[c] - tolerance
        max_val = target_color[c] + tolerance
        channel_region = (object_mask[:,:,c] >= min_val) & (object_mask[:,:,c] <= max_val)
        match_region &= channel_region

    if match_region.sum() != 0:
        return match_region
    else:
        return None
    
# Swap colors method
def swap_color(imgarray, source, dest):
    matched_color = match_color(imgarray, [source.R, source.G, source.B])
    imgarray[:,:,:3][matched_color] = [dest.R, dest.G, dest.B]
    return np.array(imgarray)


ModuleNotFoundError: No module named 'PIL'

Connect to the game
===================
Load unrealcv python client, do :code:`pip install unrealcv` first.



In [2]:
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)

INFO:__init__:204:Got connection confirm: 'connected to NeighborhoodCV'


In [3]:
# Make sure the connection works well

res = client.request('vget /unrealcv/status')
# The image resolution and port is configured in the config file.
print(res)

Is Listening
Client Connected
9000
Configuration
Config file: C:/Program Files/Epic Games/UE_4.13/Engine/Binaries/Win64/unrealcv.ini
Port: 9000
Width: 1440
Height: 900



Get objects
======================
Write a json file with the object and their corresponding classes.



In [4]:
scene_objects = client.request('vget /objects').split(' ')
print('Number of objects in this scene:', len(scene_objects))

if '257' in scene_objects:
    scene_objects.remove('257')

obj_id_to_class = {}
for obj_id in scene_objects:
    obj_id_parts = obj_id.split('_')
    class_name = obj_id_parts[0]    
    obj_id_to_class[obj_id] = class_name

# Write JSON file
with open('../data/neighborhood_object_ids.json', 'w') as outfile:
    json.dump(obj_id_to_class, outfile)

Number of objects in this scene: 3762


Get object colors
======================
First we create the color object class

In [5]:
class Color(object):
    ''' A utility class to parse color value '''
    regexp = re.compile('\(R=(.*),G=(.*),B=(.*),A=(.*)\)')
    def __init__(self, color_str):
        self.color_str = color_str
        match = self.regexp.match(color_str)
        (self.R, self.G, self.B, self.A) = [int(match.group(i)) for i in range(1,5)]

    def __repr__(self):
        return self.color_str

Then, we either load from json

In [6]:
id2color = {}
with open('../data/id2color.json') as data_file:
    data = json.load(data_file)

for obj_id in data.keys():
    color_map = data[obj_id]
    color_str = '(R=' + str(color_map['R']) + ',G=' + \
                str(color_map['G']) + ',B=' + str(color_map['B']) + \
                ',A=' + str(color_map['A']) + ')'
    color = Color(color_str)
    id2color[obj_id] = color

or We load from the scene

In [7]:
id2color = {} # Map from object id to the labeling color
for i, obj_id in enumerate(scene_objects):
    color = Color(client.request('vget /object/%s/color' % obj_id))
    id2color[obj_id] = color
    print('%d. %s : %s' % (i, obj_id, str(color)))

0. WorldSettings_1 : (R=0,G=0,B=0,A=255)
1. InstancedFoliageActor_0 : (R=0,G=0,B=255,A=255)
2. SkySphere : (R=0,G=255,B=0,A=255)
3. Landscape_0 : (R=0,G=255,B=255,A=255)
4. Road_2 : (R=255,G=0,B=0,A=255)
5. Road_3 : (R=255,G=0,B=255,A=255)
6. Road_5 : (R=255,G=255,B=0,A=255)
7. Road_7 : (R=255,G=255,B=255,A=255)
8. Road_9 : (R=0,G=0,B=127,A=255)
9. Road_10 : (R=0,G=255,B=127,A=255)
10. Road_11 : (R=255,G=0,B=127,A=255)
11. Road_12 : (R=255,G=255,B=127,A=255)
12. Road_13 : (R=0,G=127,B=0,A=255)
13. Road_14 : (R=0,G=127,B=255,A=255)
14. Road_15 : (R=255,G=127,B=0,A=255)
15. Road_17 : (R=255,G=127,B=255,A=255)
16. Road_18 : (R=0,G=127,B=127,A=255)
17. Road_19 : (R=255,G=127,B=127,A=255)
18. Road_20 : (R=127,G=0,B=0,A=255)
19. Road_21 : (R=127,G=0,B=255,A=255)
20. Road_22 : (R=127,G=255,B=0,A=255)
21. Road_23 : (R=127,G=255,B=255,A=255)
22. Road_24 : (R=127,G=0,B=127,A=255)
23. Road_25 : (R=127,G=255,B=127,A=255)
24. Road_26 : (R=127,G=127,B=0,A=255)
25. Road_27 : (R=127,G=127,B=255,A=255)

KeyboardInterrupt: 

Write to JSON if loaded from scene

In [ ]:
# Convert to serializable json dictionary
serializable_map = {}
for color_id in id2color.keys():
    curr_color = id2color[color_id]
    color_map = {}
    color_map['R'] = curr_color.R
    color_map['G'] = curr_color.G
    color_map['B'] = curr_color.B
    color_map['A'] = curr_color.A
    serializable_map[color_id] = color_map

# Write to JSON
with open('../data/id2color.json', 'w') as outfile:
    json.dump(serializable_map, outfile)

In [7]:
# Map classes to lists of objects
classes = {}

for obj_id in obj_id_to_class.keys():
    
    curr_class = obj_id_to_class[obj_id]
    if curr_class not in classes:
        classes[curr_class] = []
    
    classes[curr_class].append(obj_id)

# Write classes to json
with open('../data/neighborhood_classes.json', 'w') as outfile:
    json.dump(classes, outfile) 
    

In [9]:
# Normalize using built in API
counter = 0
for curr_class in classes.keys():
    
    
    object_list = classes[curr_class]
    curr_color = id2color[object_list[0]]
    
    for obj_id in object_list:
        
        client.request('vset /object/' + obj_id + '/color ' + \
                       str(curr_color.R) + ' ' + str(curr_color.G) + \
                       ' ' + str(curr_color.B))
        
        print(str(counter) + '. vset /object/' + obj_id + '/color ' + \
                       str(curr_color.R) + ' ' + str(curr_color.G) + \
                       ' ' + str(curr_color.B))
    
        counter += 1
    

0. vset /object/Fir_01_476/color 95 31 95
1. vset /object/Fir_19/color 95 31 95
2. vset /object/Fir_18/color 95 31 95
3. vset /object/Fir_17/color 95 31 95
4. vset /object/Fir_16/color 95 31 95
5. vset /object/Fir_15/color 95 31 95
6. vset /object/Fir_12/color 95 31 95
7. vset /object/Fir_11/color 95 31 95
8. vset /object/Fir_10/color 95 31 95
9. vset /object/Fir_139/color 95 31 95
10. vset /object/Fir_482/color 95 31 95
11. vset /object/Fir_226/color 95 31 95
12. vset /object/Fir_44/color 95 31 95
13. vset /object/Fir_45/color 95 31 95
14. vset /object/Fir_46/color 95 31 95
15. vset /object/Fir_47/color 95 31 95
16. vset /object/Fir_40/color 95 31 95
17. vset /object/Fir_41/color 95 31 95
18. vset /object/Fir_42/color 95 31 95
19. vset /object/Fir_43/color 95 31 95
20. vset /object/Fir_492/color 95 31 95
21. vset /object/Fir_223/color 95 31 95
22. vset /object/Fir_229/color 95 31 95
23. vset /object/Fir_183/color 95 31 95
24. vset /object/Fir_235/color 95 31 95
25. vset /object/Fir_23

159. vset /object/Inner_Wall_Close_Gap_90/color 127 191 207
160. vset /object/Inner_Wall_Close_Gap_36/color 127 191 207
161. vset /object/Inner_Wall_Close_Gap_38/color 127 191 207
162. vset /object/Inner_Wall_Close_Gap_198/color 127 191 207
163. vset /object/Inner_Wall_Half_Arch_102/color 127 191 207
164. vset /object/Inner_Wall_Close_Gap_351/color 127 191 207
165. vset /object/Inner_Wall_Quart_Arch_160/color 127 191 207
166. vset /object/Inner_Wall_Half_Door_19/color 127 191 207
167. vset /object/Inner_Wall_Half_Door_17/color 127 191 207
168. vset /object/Inner_Wall_Half_Door_11/color 127 191 207
169. vset /object/Inner_Wall_Close_Gap_160/color 127 191 207
170. vset /object/Inner_Wall_Close_Gap_257/color 127 191 207
171. vset /object/Inner_Wall_Close_Gap_254/color 127 191 207
172. vset /object/Inner_Door_Swing_01_C_41/color 127 191 207
173. vset /object/Inner_Door_Swing_01_C_40/color 127 191 207
174. vset /object/Inner_Wall_8th_01_163/color 127 191 207
175. vset /object/Inner_Wall_Clo

312. vset /object/Hedge_393/color 95 191 175
313. vset /object/Hedge_392/color 95 191 175
314. vset /object/Hedge_390/color 95 191 175
315. vset /object/Hedge_397/color 95 191 175
316. vset /object/Hedge_396/color 95 191 175
317. vset /object/Hedge_395/color 95 191 175
318. vset /object/Hedge_394/color 95 191 175
319. vset /object/Hedge_496/color 95 191 175
320. vset /object/Hedge_19/color 95 191 175
321. vset /object/Hedge_18/color 95 191 175
322. vset /object/Hedge_15/color 95 191 175
323. vset /object/Hedge_14/color 95 191 175
324. vset /object/Hedge_17/color 95 191 175
325. vset /object/Hedge_16/color 95 191 175
326. vset /object/Hedge_11/color 95 191 175
327. vset /object/Hedge_10/color 95 191 175
328. vset /object/Hedge_12/color 95 191 175
329. vset /object/Hedge_492/color 95 191 175
330. vset /object/Hedge_498/color 95 191 175
331. vset /object/Hedge_499/color 95 191 175
332. vset /object/Hedge_252/color 95 191 175
333. vset /object/Hedge_338/color 95 191 175
334. vset /object/H

495. vset /object/Hedge_259/color 95 191 175
496. vset /object/Hedge_618/color 95 191 175
497. vset /object/Hedge_631/color 95 191 175
498. vset /object/Hedge_635/color 95 191 175
499. vset /object/Hedge_638/color 95 191 175
500. vset /object/Hedge_596/color 95 191 175
501. vset /object/Hedge_03_135/color 95 191 175
502. vset /object/Hedge_107/color 95 191 175
503. vset /object/Hedge_104/color 95 191 175
504. vset /object/Hedge_105/color 95 191 175
505. vset /object/Hedge_102/color 95 191 175
506. vset /object/Hedge_103/color 95 191 175
507. vset /object/Hedge_101/color 95 191 175
508. vset /object/Hedge_108/color 95 191 175
509. vset /object/Hedge_811/color 95 191 175
510. vset /object/Hedge_209/color 95 191 175
511. vset /object/Hedge_208/color 95 191 175
512. vset /object/Hedge_461/color 95 191 175
513. vset /object/Hedge_205/color 95 191 175
514. vset /object/Hedge_703/color 95 191 175
515. vset /object/Hedge_357/color 95 191 175
516. vset /object/Hedge_355/color 95 191 175
517. vs

678. vset /object/Hedge_264/color 95 191 175
679. vset /object/Hedge_263/color 95 191 175
680. vset /object/Hedge_90/color 95 191 175
681. vset /object/Hedge_93/color 95 191 175
682. vset /object/Hedge_92/color 95 191 175
683. vset /object/Hedge_99/color 95 191 175
684. vset /object/Hedge_98/color 95 191 175
685. vset /object/Hedge_269/color 95 191 175
686. vset /object/Hedge_268/color 95 191 175
687. vset /object/Hedge_79/color 95 191 175
688. vset /object/Hedge_78/color 95 191 175
689. vset /object/Hedge_73/color 95 191 175
690. vset /object/Hedge_72/color 95 191 175
691. vset /object/Hedge_70/color 95 191 175
692. vset /object/Hedge_76/color 95 191 175
693. vset /object/Hedge_75/color 95 191 175
694. vset /object/Hedge_74/color 95 191 175
695. vset /object/Hedge_291/color 95 191 175
696. vset /object/Hedge_483/color 95 191 175
697. vset /object/Hedge_449/color 95 191 175
698. vset /object/Hedge_448/color 95 191 175
699. vset /object/Hedge_447/color 95 191 175
700. vset /object/Hedge

...

3000. vset /object/Wall_Low_End_37/color 15 15 255
3001. vset /object/Wall_Low_End_62/color 15 15 255
3002. vset /object/Wall_Low_Quart_122/color 15 15 255
3003. vset /object/Wall_Low_End_111/color 15 15 255
3004. vset /object/Wall_Low_8th_368/color 15 15 255
3005. vset /object/Wall_Low_End_01_243/color 15 15 255
3006. vset /object/Wall_Low_227/color 15 15 255
3007. vset /object/Wall_Low_223/color 15 15 255
3008. vset /object/Wall_Low_End_61/color 15 15 255
3009. vset /object/Wall_Low_159/color 15 15 255
3010. vset /object/Wall_Low_53/color 15 15 255
3011. vset /object/Wall_Low_51/color 15 15 255
3012. vset /object/Wall_Low_End_375/color 15 15 255
3013. vset /object/Wall_Low_55/color 15 15 255
3014. vset /object/Wall_Low_Quart_33/color 15 15 255
3015. vset /object/Wall_Low_Quart_182/color 15 15 255
3016. vset /object/House_Base_Quart_96/color 255 111 191
3017. vset /object/House_Base_Quart_62/color 255 111 191
3018. vset /object/House_Base_Quart_32/color 255 111 191
3019. vset /object/

3154. vset /object/Floor__8th_8th_2/color 223 175 31
3155. vset /object/Floor__8th_8th_3/color 223 175 31
3156. vset /object/Floor__8th_8th_7/color 223 175 31
3157. vset /object/Floor_01_Basic25/color 223 175 31
3158. vset /object/Floor_01_Basic14/color 223 175 31
3159. vset /object/Floor_01_Basic29/color 223 175 31
3160. vset /object/Floor_01_Basic28/color 223 175 31
3161. vset /object/Floor_Quart_167/color 223 175 31
3162. vset /object/Floor_Half_Half_2/color 223 175 31
3163. vset /object/Floor_Half_Half_6/color 223 175 31
3164. vset /object/Floor_Half_Half_7/color 223 175 31
3165. vset /object/Floor_Half_Half_4/color 223 175 31
3166. vset /object/Floor_16/color 223 175 31
3167. vset /object/Floor_Quart_5/color 223 175 31
3168. vset /object/Floor_10/color 223 175 31
3169. vset /object/Floor_8/color 223 175 31
3170. vset /object/Floor_9/color 223 175 31
3171. vset /object/Floor_6/color 223 175 31
3172. vset /object/Floor_7/color 223 175 31
3173. vset /object/Floor_2/color 223 175 31
3

3312. vset /object/Garden_Chair_01_4/color 223 47 47
3313. vset /object/Garden_Tressel_29/color 223 47 47
3314. vset /object/Garden_Tressel_20/color 223 47 47
3315. vset /object/Garden_Tressel_23/color 223 47 47
3316. vset /object/Garden_Rocks_12/color 223 47 47
3317. vset /object/Garden_Rocks_11/color 223 47 47
3318. vset /object/Garden_Chair_47/color 223 47 47
3319. vset /object/Garden_Chair_40/color 223 47 47
3320. vset /object/Garden_Chair_2/color 223 47 47
3321. vset /object/Garden_Chair_6/color 223 47 47
3322. vset /object/Garden_Chair_4/color 223 47 47
3323. vset /object/Garden_Chair_5/color 223 47 47
3324. vset /object/Garden_Chair_50/color 223 47 47
3325. vset /object/Garden_Rocks_8/color 223 47 47
3326. vset /object/Garden_Tressel_35/color 223 47 47
3327. vset /object/Garden_Tressel_38/color 223 47 47
3328. vset /object/Garden_Tressel_182/color 223 47 47
3329. vset /object/Garden_Tressel_73/color 223 47 47
3330. vset /object/Garden_Tressel_76/color 223 47 47
3331. vset /objec

3479. vset /object/Shutters_20/color 143 175 63
3480. vset /object/Shutters_21/color 143 175 63
3481. vset /object/Shutters_22/color 143 175 63
3482. vset /object/Shutters_28/color 143 175 63
3483. vset /object/Shutters_29/color 143 175 63
3484. vset /object/Shutters_9/color 143 175 63
3485. vset /object/Shutters_8/color 143 175 63
3486. vset /object/Shutters_5/color 143 175 63
3487. vset /object/Shutters_4/color 143 175 63
3488. vset /object/Shutters_7/color 143 175 63
3489. vset /object/Shutters_6/color 143 175 63
3490. vset /object/Shutters_3/color 143 175 63
3491. vset /object/Shutters_2/color 143 175 63
3492. vset /object/Shutters_120/color 143 175 63
3493. vset /object/Shutters_19/color 143 175 63
3494. vset /object/Shutters_18/color 143 175 63
3495. vset /object/Shutters_11/color 143 175 63
3496. vset /object/Shutters_10/color 143 175 63
3497. vset /object/Shutters_13/color 143 175 63
3498. vset /object/Shutters_12/color 143 175 63
3499. vset /object/Shutters_14/color 143 175 63

3641. vset /object/Roof_Half_5/color 79 255 63
3642. vset /object/Roof_Half_01_71/color 79 255 63
3643. vset /object/Roof_Edge_End_31/color 79 255 63
3644. vset /object/Roof_Middle_Half_Half_218/color 79 255 63
3645. vset /object/Roof_Open_9/color 79 255 63
3646. vset /object/Roof_Open_3/color 79 255 63
3647. vset /object/Roof_Open_2/color 79 255 63
3648. vset /object/Roof_Apex_332/color 79 255 63
3649. vset /object/Roof_Corner_6/color 79 255 63
3650. vset /object/Roof_Corner_7/color 79 255 63
3651. vset /object/Roof_End_Thin_3/color 79 255 63
3652. vset /object/Roof_End_Thin_01_4/color 79 255 63
3653. vset /object/Roof_End_Thin_8/color 79 255 63
3654. vset /object/Roof_End_Thin_9/color 79 255 63
3655. vset /object/Roof_148/color 79 255 63
3656. vset /object/Roof_Flat_01_474/color 79 255 63
3657. vset /object/Roof_01_122/color 79 255 63
3658. vset /object/Roof_Middle_Apex_19/color 79 255 63
3659. vset /object/Roof_Middle_Apex_8/color 79 255 63
3660. vset /object/Roof_Middle_Apex_9/colo

Begin Data Collection (Without Normalization)
======

In [10]:
with open('../data/finalTopClassesToColor.json', 'r') as infile:
    top20tocolor = json.load(infile)

top20toint = {}
for cnt, cls in enumerate(top20tocolor.keys()):
    top20toint[cls] = cnt + 1

top20toint['Oak'] = top20toint['Fir']
top20toint['Birch'] = top20toint['Fir']
top20toint['Tree'] = top20toint['Fir']

with open('../data/topClassesToInt.json', 'w') as outfile:
    json.dump(top20toint, outfile)

for batch in range(1,1001):

    # Get random location
    z = 300
    x = random.randint(-5500, 5500)
    y = random.randint(-5500, 5500)
    
    # Get random yaw
    yaw = random.randint(0,360)

    # Coordinates x, y, z
    client.request('vset /camera/0/location ' + str(x) + ' ' + str(y) + \
                   ' ' + str(z)) 

    # Get 10 shots in a series
    angles = []
    a = 0
    while len(angles) < 20:
        angles.append(a)
        a -= 3

    # Increment height sequentially
    heights = []
    height = 300
    while len(heights) < 20:
        heights.append(height)
        height += 50

    for i,angle in enumerate(angles):
        
        print("Round: " + str(batch) + " , Image: " + str(i))
        
        # x, y, z
        client.request('vset /camera/0/location ' + str(x) + ' ' + str(y) + \
                       ' ' + str(heights[i])) 

        # Pitch, yaw, roll
        client.request('vset /camera/0/rotation ' + str(angle) + ' ' + str(yaw) + ' 0')

        # Get Ground Truth
        res = client.request('vget /camera/0/object_mask png')
        object_mask = read_png(res)
        segmentation_image = Image.fromarray(object_mask)
        
        directory = 'D:/dataset_randomyaw/batch7/round' + str(batch) + '/'
        if not os.path.exists(directory):
            os.makedirs(directory)
                    
        segmentation_image.save('D:/dataset_randomyaw/batch7/round' + str(batch) + '/seg' + \
                            str(i) + '.png')
        
        print('D:/dataset_randomyaw/batch7/round' + str(batch) + '/seg' + \
                            str(i) + '.png')
        
        res = client.request('vget /camera/0/lit png')
        normal = read_png(res)
        normal = Image.fromarray(normal)
        
       
        normal.save('D:/dataset_randomyaw/batch7/round' + str(batch) + '/pic' + \
                            str(i) + '.png')
    
        
        print("Images written. ")

Round: 1 , Image: 0
D:/segmentation_data/batch6/round1/seg0.png
Images written. 
Round: 1 , Image: 1
D:/segmentation_data/batch6/round1/seg1.png
Images written. 
Round: 1 , Image: 2
D:/segmentation_data/batch6/round1/seg2.png
Images written. 
Round: 1 , Image: 3
D:/segmentation_data/batch6/round1/seg3.png
Images written. 
Round: 1 , Image: 4
D:/segmentation_data/batch6/round1/seg4.png
Images written. 
Round: 1 , Image: 5
D:/segmentation_data/batch6/round1/seg5.png
Images written. 
Round: 1 , Image: 6
D:/segmentation_data/batch6/round1/seg6.png
Images written. 
Round: 1 , Image: 7
D:/segmentation_data/batch6/round1/seg7.png
Images written. 
Round: 1 , Image: 8
D:/segmentation_data/batch6/round1/seg8.png
Images written. 
Round: 1 , Image: 9
D:/segmentation_data/batch6/round1/seg9.png
Images written. 
Round: 1 , Image: 10
D:/segmentation_data/batch6/round1/seg10.png
Images written. 
Round: 1 , Image: 11
D:/segmentation_data/batch6/round1/seg11.png
Images written. 
Round: 1 , Image: 12
D:/

Round: 6 , Image: 0
D:/segmentation_data/batch6/round6/seg0.png
Images written. 
Round: 6 , Image: 1
D:/segmentation_data/batch6/round6/seg1.png
Images written. 
Round: 6 , Image: 2
D:/segmentation_data/batch6/round6/seg2.png
Images written. 
Round: 6 , Image: 3
D:/segmentation_data/batch6/round6/seg3.png
Images written. 
Round: 6 , Image: 4
D:/segmentation_data/batch6/round6/seg4.png
Images written. 
Round: 6 , Image: 5
D:/segmentation_data/batch6/round6/seg5.png
Images written. 
Round: 6 , Image: 6
D:/segmentation_data/batch6/round6/seg6.png
Images written. 
Round: 6 , Image: 7
D:/segmentation_data/batch6/round6/seg7.png
Images written. 
Round: 6 , Image: 8
D:/segmentation_data/batch6/round6/seg8.png
Images written. 
Round: 6 , Image: 9
D:/segmentation_data/batch6/round6/seg9.png
Images written. 
Round: 6 , Image: 10
D:/segmentation_data/batch6/round6/seg10.png
Images written. 
Round: 6 , Image: 11
D:/segmentation_data/batch6/round6/seg11.png
Images written. 
Round: 6 , Image: 12
D:/

Images written. 
Round: 11 , Image: 0
D:/segmentation_data/batch6/round11/seg0.png
Images written. 
Round: 11 , Image: 1
D:/segmentation_data/batch6/round11/seg1.png
Images written. 
Round: 11 , Image: 2
D:/segmentation_data/batch6/round11/seg2.png
Images written. 
Round: 11 , Image: 3
D:/segmentation_data/batch6/round11/seg3.png
Images written. 
Round: 11 , Image: 4
D:/segmentation_data/batch6/round11/seg4.png
Images written. 
Round: 11 , Image: 5
D:/segmentation_data/batch6/round11/seg5.png
Images written. 
Round: 11 , Image: 6
D:/segmentation_data/batch6/round11/seg6.png
Images written. 
Round: 11 , Image: 7
D:/segmentation_data/batch6/round11/seg7.png
Images written. 
Round: 11 , Image: 8
D:/segmentation_data/batch6/round11/seg8.png
Images written. 
Round: 11 , Image: 9
D:/segmentation_data/batch6/round11/seg9.png
Images written. 
Round: 11 , Image: 10
D:/segmentation_data/batch6/round11/seg10.png
Images written. 
Round: 11 , Image: 11
D:/segmentation_data/batch6/round11/seg11.png


Images written. 
Round: 15 , Image: 18
D:/segmentation_data/batch6/round15/seg18.png
Images written. 
Round: 15 , Image: 19
D:/segmentation_data/batch6/round15/seg19.png
Images written. 
Round: 16 , Image: 0
D:/segmentation_data/batch6/round16/seg0.png
Images written. 
Round: 16 , Image: 1
D:/segmentation_data/batch6/round16/seg1.png
Images written. 
Round: 16 , Image: 2
D:/segmentation_data/batch6/round16/seg2.png
Images written. 
Round: 16 , Image: 3
D:/segmentation_data/batch6/round16/seg3.png
Images written. 
Round: 16 , Image: 4
D:/segmentation_data/batch6/round16/seg4.png
Images written. 
Round: 16 , Image: 5
D:/segmentation_data/batch6/round16/seg5.png
Images written. 
Round: 16 , Image: 6
D:/segmentation_data/batch6/round16/seg6.png
Images written. 
Round: 16 , Image: 7
D:/segmentation_data/batch6/round16/seg7.png
Images written. 
Round: 16 , Image: 8
D:/segmentation_data/batch6/round16/seg8.png
Images written. 
Round: 16 , Image: 9
D:/segmentation_data/batch6/round16/seg9.png


Images written. 
Round: 20 , Image: 16
D:/segmentation_data/batch6/round20/seg16.png
Images written. 
Round: 20 , Image: 17
D:/segmentation_data/batch6/round20/seg17.png
Images written. 
Round: 20 , Image: 18
D:/segmentation_data/batch6/round20/seg18.png
Images written. 
Round: 20 , Image: 19
D:/segmentation_data/batch6/round20/seg19.png
Images written. 
Round: 21 , Image: 0
D:/segmentation_data/batch6/round21/seg0.png
Images written. 
Round: 21 , Image: 1
D:/segmentation_data/batch6/round21/seg1.png
Images written. 
Round: 21 , Image: 2
D:/segmentation_data/batch6/round21/seg2.png
Images written. 
Round: 21 , Image: 3
D:/segmentation_data/batch6/round21/seg3.png
Images written. 
Round: 21 , Image: 4
D:/segmentation_data/batch6/round21/seg4.png
Images written. 
Round: 21 , Image: 5
D:/segmentation_data/batch6/round21/seg5.png
Images written. 
Round: 21 , Image: 6
D:/segmentation_data/batch6/round21/seg6.png
Images written. 
Round: 21 , Image: 7
D:/segmentation_data/batch6/round21/seg7.

...

Images written. 
Round: 543 , Image: 6
D:/segmentation_data/batch6/round543/seg6.png
Images written. 
Round: 543 , Image: 7
D:/segmentation_data/batch6/round543/seg7.png
Images written. 
Round: 543 , Image: 8
D:/segmentation_data/batch6/round543/seg8.png
Images written. 
Round: 543 , Image: 9
D:/segmentation_data/batch6/round543/seg9.png
Images written. 
Round: 543 , Image: 10
D:/segmentation_data/batch6/round543/seg10.png
Images written. 
Round: 543 , Image: 11
D:/segmentation_data/batch6/round543/seg11.png
Images written. 
Round: 543 , Image: 12
D:/segmentation_data/batch6/round543/seg12.png
Images written. 
Round: 543 , Image: 13
D:/segmentation_data/batch6/round543/seg13.png
Images written. 
Round: 543 , Image: 14
D:/segmentation_data/batch6/round543/seg14.png
Images written. 
Round: 543 , Image: 15
D:/segmentation_data/batch6/round543/seg15.png
Images written. 
Round: 543 , Image: 16
D:/segmentation_data/batch6/round543/seg16.png
Images written. 
Round: 543 , Image: 17
D:/segmenta

D:/segmentation_data/batch6/round548/seg1.png
Images written. 
Round: 548 , Image: 2
D:/segmentation_data/batch6/round548/seg2.png
Images written. 
Round: 548 , Image: 3
D:/segmentation_data/batch6/round548/seg3.png
Images written. 
Round: 548 , Image: 4
D:/segmentation_data/batch6/round548/seg4.png
Images written. 
Round: 548 , Image: 5
D:/segmentation_data/batch6/round548/seg5.png
Images written. 
Round: 548 , Image: 6
D:/segmentation_data/batch6/round548/seg6.png
Images written. 
Round: 548 , Image: 7
D:/segmentation_data/batch6/round548/seg7.png
Images written. 
Round: 548 , Image: 8
D:/segmentation_data/batch6/round548/seg8.png
Images written. 
Round: 548 , Image: 9
D:/segmentation_data/batch6/round548/seg9.png
Images written. 
Round: 548 , Image: 10
D:/segmentation_data/batch6/round548/seg10.png
Images written. 
Round: 548 , Image: 11
D:/segmentation_data/batch6/round548/seg11.png
Images written. 
Round: 548 , Image: 12
D:/segmentation_data/batch6/round548/seg12.png
Images writte

Images written. 
Round: 552 , Image: 17
D:/segmentation_data/batch6/round552/seg17.png
Images written. 
Round: 552 , Image: 18
D:/segmentation_data/batch6/round552/seg18.png
Images written. 
Round: 552 , Image: 19
D:/segmentation_data/batch6/round552/seg19.png
Images written. 
Round: 553 , Image: 0
D:/segmentation_data/batch6/round553/seg0.png
Images written. 
Round: 553 , Image: 1
D:/segmentation_data/batch6/round553/seg1.png
Images written. 
Round: 553 , Image: 2
D:/segmentation_data/batch6/round553/seg2.png
Images written. 
Round: 553 , Image: 3
D:/segmentation_data/batch6/round553/seg3.png
Images written. 
Round: 553 , Image: 4
D:/segmentation_data/batch6/round553/seg4.png
Images written. 
Round: 553 , Image: 5
D:/segmentation_data/batch6/round553/seg5.png
Images written. 
Round: 553 , Image: 6
D:/segmentation_data/batch6/round553/seg6.png
Images written. 
Round: 553 , Image: 7
D:/segmentation_data/batch6/round553/seg7.png
Images written. 
Round: 553 , Image: 8
D:/segmentation_data

D:/segmentation_data/batch6/round557/seg12.png
Images written. 
Round: 557 , Image: 13
D:/segmentation_data/batch6/round557/seg13.png
Images written. 
Round: 557 , Image: 14
D:/segmentation_data/batch6/round557/seg14.png
Images written. 
Round: 557 , Image: 15
D:/segmentation_data/batch6/round557/seg15.png
Images written. 
Round: 557 , Image: 16
D:/segmentation_data/batch6/round557/seg16.png
Images written. 
Round: 557 , Image: 17
D:/segmentation_data/batch6/round557/seg17.png
Images written. 
Round: 557 , Image: 18
D:/segmentation_data/batch6/round557/seg18.png
Images written. 
Round: 557 , Image: 19
D:/segmentation_data/batch6/round557/seg19.png
Images written. 
Round: 558 , Image: 0
D:/segmentation_data/batch6/round558/seg0.png
Images written. 
Round: 558 , Image: 1
D:/segmentation_data/batch6/round558/seg1.png
Images written. 
Round: 558 , Image: 2
D:/segmentation_data/batch6/round558/seg2.png
Images written. 
Round: 558 , Image: 3
D:/segmentation_data/batch6/round558/seg3.png
Imag

KeyboardInterrupt: 

Begin Data Collection (With Normalization)
======

In [13]:
for batch in range(1,5):

    # Get random location
    z = 300
    x = random.randint(-5000, 5000)
    y = random.randint(-5000, 5000)

    # Coordinates x, y, z
    client.request('vset /camera/0/location ' + str(x) + ' ' + str(y) + \
                   ' ' + str(z)) 

    # Get 10 shots in a series
    angles = []
    a = 0
    while len(angles) < 20:
        angles.append(a)
        a -= 3

    # Increment height sequentially
    heights = []
    height = 300
    while len(heights) < 20:
        heights.append(height)
        height += 50

    for i,angle in enumerate(angles):
        
        print("Batch: " + str(batch) + " , Image: " + str(i))
        
        # x, y, z
        client.request('vset /camera/0/location ' + str(x) + ' ' + str(y) + \
                       ' ' + str(heights[i])) 

        # Pitch, yaw, roll
        client.request('vset /camera/0/rotation ' + str(angle) + ' 0 0')

        """
        res = client.request('vget /camera/0/object_mask ./batch/round' + \
                             str(batch) + '/lit' + str(i) + '.png')
        
        print('The image is saved to %s' % res)
        """
        
        # Get Ground Truth
        res = client.request('vget /camera/0/object_mask png')
        object_mask = read_png(res)
        
        # Get all classes in image
        print("Getting all object_ids in image....")
        id2mask = {}
        for obj_id in scene_objects:
            color = id2color[obj_id]
            mask = match_color(object_mask, [color.R, color.G, color.B], tolerance = 3)
            if mask is not None:
                id2mask[obj_id] = mask
        
        print("Got all object_ids in image.")

        """
        Group classes together
        """ 
        class_groups = {}

        # Go through the matched objects
        for idmask in id2mask.keys():

            # Get the class from the neighborhood classes object
            curr_class = obj_id_to_class[idmask]

            # If the class is not in the class_groups map, add it
            if curr_class not in class_groups:
                class_groups[curr_class] = [] 

            # Add the idmask to it's corresponding class
            class_groups[curr_class].append(idmask)


        # Normalize by class
        for cls in class_groups.keys():
            class_base = class_groups[cls][0]
            class_color = id2color[class_base]
            for class_id in class_groups[cls]:
                current_color = id2color[class_id]
                object_mask = swap_color(object_mask, current_color, class_color) 
        
        # Write to file
        print(object_mask.shape)
        normalized_img = Image.fromarray(object_mask)

        directory = './batch/round' + str(batch) + '/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        normalized_img.save('./batch/round' + str(batch) + '/seg' + \
                            str(i) + '.png')
        
        res = client.request('vget /camera/0/lit png')
        normal = read_png(res)
        print(normal.shape)
        normal = Image.fromarray(normal)
        
        normal.save('./batch/round' + str(batch) + '/pic' + \
                            str(i) + '.png')
        
        print("Images written. ")


Batch: 1 , Image: 0
Getting all object_ids in image....
Got all object_ids in image.
(900L, 1440L, 4L)
(900L, 1440L, 4L)
Images written. 
Batch: 1 , Image: 1
Getting all object_ids in image....


KeyboardInterrupt: 